<a href="https://colab.research.google.com/github/suvidsahay/PersonalizingLLM/blob/master/personalise_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvi

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import json
from datasets import Dataset, DatasetDict

In [ ]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def create_dataset(questions, outputs):
  # Convert outputs to a dictionary for fast lookup
  output_dict = {output['id']: output['output'] for output in outputs['golds']}

  # Merge the questions and outputs based on the 'id'
  merged_data = []
  for question in questions:
      question_id = question['id']
      if question_id in output_dict:
          merged_entry = {
              "id": question_id,
              "input": question['input'],
              "profile": question['profile'],
              "output": output_dict[question_id]
          }
          merged_data.append(merged_entry)
  merged_data = merged_data[:500]

  # Convert the merged data to a Hugging Face Dataset
  dataset = Dataset.from_dict({"id": [item["id"] for item in merged_data],
                              "input": [item["input"] for item in merged_data],
                              "profile": [item["profile"] for item in merged_data],
                              "output": [item["output"] for item in merged_data]})

  return dataset


In [ ]:
# Acquire the training data from Hugging Face
# Load the JSON files
with open('train_questions.json', 'r') as f:
    questions_train = json.load(f)

with open('train_outputs.json', 'r') as f:
    outputs_train = json.load(f)

with open('validation_questions.json', 'r') as f:
    questions_val = json.load(f)

with open('validation_outputs.json', 'r') as f:
    outputs_val = json.load(f)

# Create a DatasetDict
dataset_dict = DatasetDict({"train": create_dataset(questions_train, outputs_train), "validation": create_dataset(questions_val, outputs_val)})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'profile', 'output'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['id', 'input', 'profile', 'output'],
        num_rows: 500
    })
})


In [ ]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def e5_retrieve(inputs, profile_texts):
  # Each input text should start with "query: " or "passage: ".
  # For tasks other than retrieval, you can simply use the "query: " prefix.

  tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-base-v2')
  model = AutoModel.from_pretrained('intfloat/e5-base-v2')
  selected_profile_texts = []

  i = 0
  # Create input texts for each query and passage
  for input_text, profiles in zip(inputs, profile_texts):
      print(i)
      input_texts = []

      query_text = f'query: {input_text}'
      input_texts.append(query_text)
      for profile in profiles:
          passage_text = f'passage: {profile}'
          input_texts.append(passage_text)
      batch_dict = tokenizer(input_texts, max_length=256, padding=True, truncation=True, return_tensors='pt')

      outputs = model(**batch_dict)
      embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

      # normalize embeddings
      embeddings = F.normalize(embeddings, p=2, dim=1)
      scores = (embeddings[:1] @ embeddings[1:].T) * 100
      top_k_indices = scores.squeeze().topk(k=5).indices
      top_k_profiles = [profiles[i] for i in top_k_indices]
      selected_profile_texts.append(top_k_profiles)
      # print(top_k_profiles)
      i += 1

  # Tokenize the input texts
  return selected_profile_texts



# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [doc for doc in examples["input"]]
   profile_texts = [[text["text"] for text in doc] for doc in examples["profile"]]
  #  profile_texts = [profile_texts[0][:3]]
   profile_texts = e5_retrieve(inputs, profile_texts)
   print(profile_texts)
   input = ["\"" + "\", and \"".join(y) + "\" are sample tweets by a person. Given the tweets, " + x for x, y in zip(inputs, profile_texts)]
   print(input)
   model_inputs = tokenizer(input, max_length=256, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["output"],
                      max_length=128,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs



In [ ]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [ ]:
# Global Parameters
L_RATE = 5e-5
BATCH_SIZE = 16
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 1e-4
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   logging_dir="./logs",  # Ensure logging
   logging_steps=10,  # Log every 10 steps
   save_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["validation"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
last_checkpoint = "./results/checkpoint-96"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

In [ ]:

with open('train_questions.json', 'r') as f:
    test_ques = json.load(f)

inputs = test_ques[0]["input"]
profile_texts = [doc["text"] for doc in test_ques[0]["profile"][:5]]

input_texts = ["\"" + "\", and \"".join(profile_texts) + "\" are sample tweets by a person. Given the tweets, " + inputs]

print(input_texts)
model_inputs = tokenizer(input_texts, return_tensors="pt")
print(len(model_inputs.input_ids[0]))

outputs = finetuned_model.generate(**model_inputs, max_length=100)
answer = tokenizer.decode(outputs[0])
from textwrap import fill

print(answer)